<a href="https://colab.research.google.com/github/pushkarraj/My_experiments/blob/main/Bloom_Inference_using_accelarate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/Paraphrasing
%ls

/content/drive/MyDrive/Colab Notebooks/Paraphrasing
bloom17bhuge50k5ep/
bloom_sharded/
cached_lm_BloomTokenizerFast_256_Master_train.txt
cached_lm_BloomTokenizerFast_256_Master_train.txt.lock
cached_lm_BloomTokenizerFast_256_Master_val.txt
cached_lm_BloomTokenizerFast_256_Master_val.txt.lock
cached_lm_GPT2Tokenizer_255_Master_train.txt
cached_lm_GPT2Tokenizer_255_Master_train.txt.lock
cached_lm_GPT2Tokenizer_255_Master_val.txt
cached_lm_GPT2Tokenizer_255_Master_val.txt.lock
gpt3_para_data1.csv
Master_Gpt3_output.csv
Master_train.txt
Master_val.txt
new.txt
requirements.txt


In [ ]:
!pip install --quiet -r requirements.txt
#!pip install --quiet datasets
!pip install --quiet git+https://github.com/huggingface/accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 87.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 101.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import os
import gc
import time
import torch
import pandas as pd
from transformers import AutoModelForCausalLM,AutoConfig,AutoTokenizer,BloomForCausalLM,pipeline
from accelerate import Accelerator

In [ ]:
def cleanup():
    gc.collect()
    torch.cuda.empty_cache()
cleanup()

In [ ]:
accelerator = Accelerator()
device = accelerator.device

In [ ]:
from transformers import AutoModel

checkpoint = "/content/drive/MyDrive/Colab Notebooks/Paraphrasing/bloom17bhuge50k5ep/model"

model = BloomForCausalLM.from_pretrained(checkpoint)

In [ ]:
#Run when you need to shard the models otherwise no need
import json
import os

if not os.path.exists('bloom_sharded'):
   os.makedirs('bloom_sharded')
path=os.getcwd()
model.save_pretrained('bloom_sharded', max_shard_size="2GB")
with open(os.path.join('bloom_sharded', "pytorch_model.bin.index.json"), "r") as f:
  index = json.load(f)

print(index.keys())

In [ ]:
import json

with open(os.path.join('bloom_sharded', "pytorch_model.bin.index.json"), "r") as f:
  index = json.load(f)

index['weight_map']

{'lm_head.weight': 'pytorch_model-00006-of-00006.bin',
 'transformer.h.0.input_layernorm.bias': 'pytorch_model-00003-of-00006.bin',
 'transformer.h.0.input_layernorm.weight': 'pytorch_model-00003-of-00006.bin',
 'transformer.h.0.mlp.dense_4h_to_h.bias': 'pytorch_model-00003-of-00006.bin',
 'transformer.h.0.mlp.dense_4h_to_h.weight': 'pytorch_model-00003-of-00006.bin',
 'transformer.h.0.mlp.dense_h_to_4h.bias': 'pytorch_model-00003-of-00006.bin',
 'transformer.h.0.mlp.dense_h_to_4h.weight': 'pytorch_model-00003-of-00006.bin',
 'transformer.h.0.post_attention_layernorm.bias': 'pytorch_model-00003-of-00006.bin',
 'transformer.h.0.post_attention_layernorm.weight': 'pytorch_model-00003-of-00006.bin',
 'transformer.h.0.self_attention.dense.bias': 'pytorch_model-00003-of-00006.bin',
 'transformer.h.0.self_attention.dense.weight': 'pytorch_model-00003-of-00006.bin',
 'transformer.h.0.self_attention.query_key_value.bias': 'pytorch_model-00003-of-00006.bin',
 'transformer.h.0.self_attention.quer

In [ ]:
from accelerate import init_empty_weights,infer_auto_device_map

checkpoint = "/content/drive/MyDrive/Colab Notebooks/Paraphrasing/bloom17bhuge50k5ep/model"
#checkpoint= "/content/drive/MyDrive/Colab Notebooks/Paraphrasing/bloom_sharded"
config = AutoConfig.from_pretrained(checkpoint,device_map="auto")

with init_empty_weights():
    model = AutoModelForCausalLM.from_config(config)

In [ ]:
device_map = infer_auto_device_map(
    model, 
    no_split_module_classes=["BloomBlock"]
)

In [ ]:
my_device_map= {'transformer.word_embeddings': 0,
 'transformer.word_embeddings_layernorm': 0,
 'transformer.h.0': 0,
 'transformer.h.1': 0,
 'transformer.h.2': 0,
 'transformer.h.3': 0,
 'transformer.h.4': 0,
 'transformer.h.5': 0,
 'transformer.h.6': 0,
 'transformer.h.7': 0,
 'transformer.h.8': 0,
 'transformer.h.9': 0,
 'transformer.h.10': 0,
 'transformer.h.11': 0,
 'transformer.h.12': 0,
 'transformer.h.13': 0,
 'transformer.h.14': 0,
 'transformer.h.15': 0,
 'transformer.h.16': 0,
 'transformer.h.17': 0,
 'transformer.h.18': 0,
 'transformer.h.19': 0,
 'transformer.h.20': 0,
 'transformer.h.21': 0,
 'transformer.h.22': 0,
 'transformer.h.23': 0,
 'transformer.ln_f': 0,
 'lm_head': 0}

In [ ]:
from accelerate import load_checkpoint_and_dispatch
t= "/content/drive/MyDrive/Colab Notebooks/Paraphrasing/bloom_sharded"
model_new = load_checkpoint_and_dispatch(model,t, device_map=my_device_map, no_split_module_classes=["BloomBlock"])

In [ ]:
model_new.hf_device_map

{'transformer.word_embeddings': 0,
 'transformer.word_embeddings_layernorm': 0,
 'transformer.h.0': 0,
 'transformer.h.1': 0,
 'transformer.h.2': 0,
 'transformer.h.3': 0,
 'transformer.h.4': 0,
 'transformer.h.5': 0,
 'transformer.h.6': 0,
 'transformer.h.7': 0,
 'transformer.h.8': 0,
 'transformer.h.9': 0,
 'transformer.h.10': 0,
 'transformer.h.11': 0,
 'transformer.h.12': 0,
 'transformer.h.13': 0,
 'transformer.h.14': 0,
 'transformer.h.15': 0,
 'transformer.h.16': 0,
 'transformer.h.17': 0,
 'transformer.h.18': 0,
 'transformer.h.19': 0,
 'transformer.h.20': 0,
 'transformer.h.21': 0,
 'transformer.h.22': 0,
 'transformer.h.23': 0,
 'transformer.ln_f': 0,
 'lm_head': 0}

In [ ]:
#model = BloomForCausalLM.from_pretrained('/content/drive/MyDrive/Colab Notebooks/Paraphrasing/bloom17bhuge50k5ep/model').to(device)
tokenizer = AutoTokenizer.from_pretrained('bigscience/bloom-1b7',truncation=True,padding=True)

In [ ]:
model = accelerator.prepare(model_new)

In [ ]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

sentence = "I am a web developer with extensive knowledge of developing, enhancing, maintaining, and redesigning websites and mobile APPS."
#sentence="I am Shiv from 'appnox technologies'."
#sentence="I am a boy"
text =  "<s>"+ sentence + "</s>>>>><p>"

encoding = tokenizer.encode_plus(text,pad_to_max_length=True, return_tensors="pt",truncation=True)
input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)

outputs = model.generate(
    input_ids=input_ids, attention_mask=attention_masks,
    max_length=256,
    temperature=0.7,
    do_sample=True,
    top_k=100,
    top_p=0.95,
    early_stopping=True,
)

for output in outputs:
    line = tokenizer.decode(output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
    #print(line)
    print(line.split(">>>><p>")[1].split("</p>")[0])

I am a web developer with significant understanding of developing, developing, improving, and redesigning websites and mobile APPS.
